In [6]:
import pandas as pd
from geopy.geocoders import Nominatim
import geopandas as gpd
from tqdm import tqdm
import folium
from shapely.geometry import Point
from scipy.spatial import KDTree
import numpy as np

In [16]:
df = pd.read_csv("/Users/bugragorkem/Desktop/Uni/5. Semester /Data Science Projekt/archive/sensor_meta_feature.csv", sep = "\t")

In [17]:
nan_analysis = df.isna().sum().to_frame(name='NaN_Count')
nan_analysis['NaN_Percentage'] = (df.isna().mean() * 100).round(2)
nan_analysis

,NaN_Count,NaN_Percentage
Fwy,0,0.00
District,0,0.00
County,0,0.00
City,5007,29.50
Abs PM,0,0.00
Length,6354,37.44
station_id,0,0.00
Name,7,0.04
Type,0,0.00
Sensor Type,4064,23.95


In [18]:
missing_indices = df[df['Lat'].isnull() | df['Lng'].isnull()].index

# Ausgabe der fehlenden Indizes
print("Indizes mit fehlenden Werten:", missing_indices.tolist())

Indizes mit fehlenden Werten: [4180, 4190, 4506, 4507, 4508, 4678, 4745, 4746, 4747, 6726, 6727, 6752, 6753, 6754, 7204, 7205, 7226, 11490, 11491, 11492, 11603, 11831, 12158, 12160, 12366, 12566]


In [19]:
df.at[12366 , 'Lat'] = 34.149537
df.at[12366, 'Lng'] = -118.155540

In [15]:
df.to_csv("/Users/bugragorkem/Desktop/Uni/5. Semester /Data Science Projekt/archive/sensor_meta_feature.csv", sep = "\t")

In [20]:
# Filtern der Zeilen, bei denen die Stadt fehlt, aber Lat und Lng vorhanden sind
missing_cities = df[df['City'].isna() & df['Lat'].notna() & df['Lng'].notna()]

# Geolocator initialisieren
geolocator = Nominatim(user_agent="city_filler")

In [25]:
def get_city(lat, lng):
    try:
        location = geolocator.reverse((lat, lng), exactly_one=True)
        if location:
            address = location.raw['address']
            
            # Stadt suchen
            if 'city' in address:
                return address['city']
            elif 'town' in address:
                return address['town']
            
    except Exception as e:
        print(f"Fehler bei der Abfrage für ({lat}, {lng}): {e}")

    # Rückgabe None, falls keine Stadt gefunden werden kann
    return None
    

In [26]:
# Fortschrittsanzeige
tqdm.pandas()

# Fehlende Städte ergänzen
df.loc[missing_cities.index, 'City'] = missing_cities.progress_apply(
    lambda row: get_city(row['Lat'], row['Lng']), axis=1
)

  1%|          | 57/4994 [01:42<2:48:23,  2.05s/it]

Fehler bei der Abfrage für (36.982849, -121.964311): HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /reverse?lat=36.982849&lon=-121.964311&format=json&addressdetails=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))


  1%|▏         | 72/4994 [02:12<3:10:40,  2.32s/it]

Fehler bei der Abfrage für (36.6672, -121.814481): HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /reverse?lat=36.6672&lon=-121.814481&format=json&addressdetails=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))


  2%|▏         | 78/4994 [02:22<2:49:13,  2.07s/it]

...

Fehler bei der Abfrage für (37.690595, -122.071617): HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /reverse?lat=37.690595&lon=-122.071617&format=json&addressdetails=1 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x132a74f40>: Failed to resolve 'nominatim.openstreetmap.org' ([Errno 8] nodename nor servname provided, or not known)"))
Fehler bei der Abfrage für (37.690589, -122.074877): HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /reverse?lat=37.690589&lon=-122.074877&format=json&addressdetails=1 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x132abec10>: Failed to resolve 'nominatim.openstreetmap.org' ([Errno 8] nodename nor servname provided, or not known)"))
Fehler bei der Abfrage für (37.690526, -122.084955): HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /reverse?la

 97%|█████████▋| 4848/4994 [1:49:25<00:00, 572.45it/s]

Fehler bei der Abfrage für (34.009472, -118.058066): HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /reverse?lat=34.009472&lon=-118.058066&format=json&addressdetails=1 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x132c1c430>: Failed to resolve 'nominatim.openstreetmap.org' ([Errno 8] nodename nor servname provided, or not known)"))
Fehler bei der Abfrage für (34.009472, -118.058066): HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /reverse?lat=34.009472&lon=-118.058066&format=json&addressdetails=1 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x132c2b7f0>: Failed to resolve 'nominatim.openstreetmap.org' ([Errno 8] nodename nor servname provided, or not known)"))
Fehler bei der Abfrage für (34.051776, -118.005858): HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /reverse?la

100%|██████████| 4994/4994 [1:49:25<00:00,  1.31s/it] 

Fehler bei der Abfrage für (38.024938, -122.11109): HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /reverse?lat=38.024938&lon=-122.11109&format=json&addressdetails=1 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x132c42610>: Failed to resolve 'nominatim.openstreetmap.org' ([Errno 8] nodename nor servname provided, or not known)"))
Fehler bei der Abfrage für (38.024938, -122.11109): HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /reverse?lat=38.024938&lon=-122.11109&format=json&addressdetails=1 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x132aedfd0>: Failed to resolve 'nominatim.openstreetmap.org' ([Errno 8] nodename nor servname provided, or not known)"))
Fehler bei der Abfrage für (38.02567, -122.112891): HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /reverse?lat=38.

In [6]:
nan_analysis = test.isna().sum().to_frame(name='NaN_Count')
nan_analysis['NaN_Percentage'] = (test.isna().mean() * 100).round(2)
nan_analysis

,NaN_Count,NaN_Percentage
Fwy,0,0.00
District,0,0.00
County,0,0.00
City,2250,13.26
Abs PM,0,0.00
Length,6354,37.44
station_id,0,0.00
Name,7,0.04
Type,0,0.00
Sensor Type,4064,23.95


In [11]:
nan_analysis = test.isna().sum().to_frame(name='NaN_Count')
nan_analysis['NaN_Percentage'] = (test.isna().mean() * 100).round(2)
nan_analysis

,NaN_Count,NaN_Percentage
Fwy,0,0.00
District,0,0.00
County,0,0.00
City,1891,11.14
Abs PM,0,0.00
Length,6354,37.44
station_id,0,0.00
Name,7,0.04
Type,0,0.00
Sensor Type,4064,23.95


In [12]:
# Erstelle eine zentrale Karte von Kalifornien
kalifornien_karte = folium.Map(location=[37.5, -119.5], zoom_start=6)

# Füge Marker für jeden Punkt mit NaN-Wert in "City" hinzu
for idx, row in nan_city.iterrows():
    lat, lng = row['Lat'], row['Lng']
    if pd.notna(lat) and pd.notna(lng):  # Stelle sicher, dass Lat/Lng-Werte existieren
        folium.Marker(
            location=[lat, lng],
            popup=f"NaN City at index: {idx}",
            icon=folium.Icon(color="red", icon="point")
        ).add_to(kalifornien_karte)


In [13]:
kalifornien_karte

In [29]:
test['geometry'] = test.apply(lambda x: Point((x['Lng'], x['Lat'])), axis=1)
verkehrs_gdf = gpd.GeoDataFrame(test, geometry='geometry', crs="EPSG:4326")